In [1]:
import pickle as pickle
import os
import pandas as pd
import torch
import sklearn
import numpy as np
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import AutoTokenizer, AutoConfig, AutoModelForSequenceClassification, AutoModelForTokenClassification, Trainer, TrainingArguments, RobertaConfig, RobertaTokenizer, RobertaForSequenceClassification, BertTokenizer
from load_data import *
from entity_model import *
from embeddings import *
import torch.nn.functional as F
from transformers.activations import ACT2FN

2022-03-29 13:35:34.524467: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0


In [2]:
def label_to_num(label):
  num_label = []
  with open('dict_label_to_num.pkl', 'rb') as f:
    dict_label_to_num = pickle.load(f)
  for v in label:
    num_label.append(dict_label_to_num[v])
  return num_label

In [3]:
#MODEL_NAME = 'klue/roberta-base'
MODEL_NAME = 'monologg/kobigbird-bert-base'
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [4]:
#MODEL_NAME = 'klue/roberta-base'
MODEL_NAME = 'monologg/kobigbird-bert-base'
tokenizer = AutoTokenizer.from_pretrained('/opt/ml/code/vocabs')
train_dataset = load_data("/opt/ml/dataset/train/train.csv")
train_label = label_to_num(train_dataset['label'].values)

tokenized_train = tokenized_dataset(train_dataset, tokenizer)

RE_train_dataset = RE_Dataset(tokenized_train, train_label)
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
model_config =  AutoConfig.from_pretrained(MODEL_NAME)
model_config.num_labels = 30


In [5]:
input_ids = tokenized_train['input_ids'][:10].to(device)
attention_mask = tokenized_train['attention_mask'][:10].to(device)
token_type_ids = tokenized_train['token_type_ids'][:10].to(device)
sub_mask = tokenized_train['sub_mask'][:10].to(device)
obj_mask = tokenized_train['obj_mask'][:10].to(device)
labels = torch.tensor(train_label[:10]).to(device)

In [ ]:
model = Entity_Embedding_Model(model_config,0.1)
model.parameters
model.to(device)

In [25]:
outputs=model(input_ids = input_ids,attention_mask=attention_mask,token_type_ids=token_type_ids,sub_mask=sub_mask, obj_mask=obj_mask,labels=labels)
outputs

Attention type 'block_sparse' is not possible if sequence_length: 238 <= num global tokens: 2 * config.block_size + min. num sliding tokens: 3 * config.block_size + config.num_random_blocks * config.block_size + additional buffer: config.num_random_blocks * config.block_size = 704 with config.block_size = 64, config.num_random_blocks = 3. Changing attention type to 'original_full'...


(tensor(3.4000, device='cuda:0', grad_fn=<NllLossBackward>),
 tensor([[ 9.5655e-02, -6.8037e-02,  1.6411e-01, -1.3122e-02,  1.7444e-01,
           7.7310e-02, -3.3407e-01, -1.6512e-01,  5.0258e-02, -6.3516e-02,
           1.8311e-01,  3.3753e-02, -1.1108e-01,  4.5100e-02,  6.2420e-02,
          -7.0726e-02, -3.2950e-02,  6.4919e-02, -1.0130e-02, -4.1956e-03,
           1.9028e-01, -6.2661e-02, -1.7565e-01,  4.2464e-02, -1.2685e-02,
          -8.9916e-02, -5.9184e-02,  6.0081e-02,  1.2473e-01, -6.0406e-04],
         [ 1.4417e-01, -1.0949e-04,  1.3923e-01, -1.5134e-02,  1.5245e-01,
           2.2181e-02, -3.2587e-01, -2.2573e-01,  5.7637e-02, -1.3350e-02,
           1.5771e-01,  8.3234e-02, -1.4571e-01,  2.4053e-02,  1.3242e-01,
          -9.5140e-02,  1.9936e-02,  7.9553e-02,  1.1668e-01, -2.3917e-03,
           1.0415e-01,  9.5021e-02, -1.7551e-01,  3.7264e-02, -1.1916e-01,
          -8.6840e-02, -1.4667e-01,  6.5819e-02,  8.0097e-02,  2.9938e-02],
         [ 7.5072e-02, -5.8560e-02,  

In [11]:
logits=outputs.last_hidden_state
logits.size()

torch.Size([10, 238, 768])

In [22]:
a=logits[1, 1, :5]
b=logits[1, 5, :5]
c=logits[1, 10, :5]
d=logits[1, 16, :5]
k=(2*a+2*b+c+d)/6
k.tolist()

[0.07206352800130844,
 0.07456713914871216,
 -0.10029201954603195,
 -0.11513102054595947,
 0.15977250039577484]

In [20]:

k.tolist().index(0.0721)

ValueError: 0.0721 is not in list

In [33]:
output_prob=[]
output_pred=[]

In [34]:
for logit in logits:
    prob = F.softmax(logit,dim=-1).detach().cpu().numpy().tolist()
    logit = logit.detach().cpu().numpy()
    result = np.argmax(logit)

    output_prob.append(prob)
    output_pred.append(result)

In [36]:
output_pred

[9, 13, 28, 23, 23, 28, 29, 28, 9, 28]